# Data pipeline

In [13]:
import cv2
import imghdr
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from matplotlib import pyplot as plt
from keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input


Crop only car from whole image

Usage: 
1. Call is_usable(img) to check if valid image.
2. If valid call detect_cars(img), this will detect car and crop that part only.
3. Will return the largest detected car.


In [9]:
def is_usable(image_path):
    return os.path.isfile(image_path)


def detect_cars(img):
    # Load the pre-trained YOLO model
    net = cv2.dnn.readNetFromDarknet("yolov3.cfg", "yolov3.weights")

    # Define the list of class labels for YOLO
    class_labels = ["car"]

    # Get the output layer names of the network
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    # Preprocess the image
    blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)

    # Pass the image through the network
    net.setInput(blob)
    layer_outputs = net.forward(output_layers)

    # Process the layer outputs to detect cars
    boxes = []
    confidences = []
    class_ids = []
    (H, W) = img.shape[:2]

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if class_id == 2 and confidence > 0.3:  # Only detect cars with confidence higher than 0.3
                box = detection[0:4] * np.array([W, H, W, H])
                (center_x, center_y, width, height) = box.astype("int")

                x = int(center_x - (width / 2))
                y = int(center_y - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maxima suppression to remove overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

    # Find the largest detected car area
    max_area = 0
    largest_car_image = None
    for i in indices:
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])
        if w > 0 and h > 0:
            area = w * h
            if area > max_area:
                max_area = area
                largest_car_image = cv2.resize(img[y:y+h, x:x+w], (400, 400))

    return largest_car_image

Crop center for logo from car image
Usage:
1. call crop_center_square(img, size), use size= 200
2. Returns cropped image.

In [ ]:
def crop_center_square(image, size):
    h, w = image.shape[:2]
    min_dim = min(h, w)
    if min_dim < size:
        raise ValueError(f"Image dimensions ({h}, {w}) are smaller than the desired size {size}")
    start_x = (w - size) // 2
    start_y = (h - size) // 2
    cropped_image = image[start_y:start_y+size, start_x:start_x+size]
    return cropped_image

# Set the desired size of the square
size = 200

In [1]:
img_path="Give Path here"

In [14]:

if is_usable(img_path):
    img = cv2.imread(image_path)
    car_image = detect_cars(img)
    if car_image is not None:
        img=car_image
    else:
        print("No car found")


In [6]:
model = load_model(r"E:\intel Internship\eAuto\Models\model_resnet50.h5")

# Preprocess for model

In [15]:
x=img.img_to_array(img)
x
x.shape
x=x/255
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

NameError: name 'img' is not defined

Predict 

In [ ]:
#run model on image
r=model.predict(img_data)
print(r)
#find max probability from array
a=np.argmax(r, axis=1)
a
#print label of predicted car
# Class labels (replace with your own class labels)
class_labels = ['Ford', 'Honday', 'Hyundai', 'Nissan', 'Renault', 'Suzuki', 'Tata', 'Toyota', 'Volkswagen']

print(class_labels[a[0]])

# Display the image
plt.imshow(img)
plt.axis('off') # Remove axis ticks and labels
plt.show()
